In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv('C:\\Users\\ashish\\Desktop\\deep\\data\\Churn_Modelling.csv')

In [4]:
X =dataset.iloc[:,3:13]

In [5]:
y= dataset.iloc[:,13]

In [6]:
geography= pd.get_dummies(X['Geography'],drop_first=True)
gender =  pd.get_dummies(X['Gender'],drop_first=True)

In [7]:
X =pd.concat([X,geography,gender],axis=1)

In [8]:
X=X.drop(['Geography','Gender'],axis=1)

In [9]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
scalar= StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test =scalar.fit_transform(X_test)

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [13]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Flatten,LeakyReLU,BatchNormalization,Dropout
from keras.activations import relu,sigmoid

In [14]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i ==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid')) 
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [15]:
model =KerasClassifier(build_fn=create_model,verbose=0)

In [16]:
layers = [[20],[40,20],[45,30,15]]
activation=['sigmoid','relu']


In [17]:
param_grid = dict(layers=layers,activation=activation,batch_size=[128,256],epochs=[30])

In [18]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5,n_jobs=-1)

In [19]:
gridresult = grid.fit(X_train,y_train)

W0312 15:18:30.275712 12040 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0312 15:18:37.953098 12040 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [20]:
print(gridresult.best_score_,gridresult.best_params_)

0.8554999947547912 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [21]:
y_pred=grid.predict(X_test)
y_pred = (y_pred>0.5)

In [22]:
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score=accuracy_score(y_test,y_pred)
confusion_matrix=confusion_matrix(y_pred,y_test)

In [23]:
accuracy_score

0.8675

In [24]:
confusion_matrix

array([[1552,  222],
       [  43,  183]], dtype=int64)